# CSE 234 Programming Assignment 3: Speculative Decoding

## Setup

In [1]:
import os
import torch
import time
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from typing import List, Tuple, Dict, Optional

/home/xunyijiang/miniconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Speculative Decoding

In [2]:
class SpeculativeDecoder:
    def __init__(self, target_model_name: str, draft_model_name: str, device: str = "cuda"):
        """
        Initialize the speculative decoder with target and draft models.

        Args:
            target_model_name: HuggingFace model ID for the larger target model.
            draft_model_name: HuggingFace model ID for the smaller draft model.
            device: Device to run models on ("cuda" or "cpu").
        """
        self.device = device
        self.target_model, self.target_tokenizer = self.initialize_target_model(target_model_name)
        self.draft_model, self.draft_tokenizer = self.initialize_draft_model(draft_model_name)

        # Ensure tokenizers are compatible
        assert self.target_tokenizer.vocab == self.draft_tokenizer.vocab, "Tokenizers must be compatible"

    def initialize_target_model(self, model_name: str):
        """Initialize the larger target model with caching enabled and proper pad token."""
        print(f"Loading target model: {model_name}")
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        # TODO: Implement target model initialization
        # 1. Set the pad token if it doesn't exist
        # 2. Load the model with appropriate settings for inference
        # 3. Enable any optimizations that might help with performance
        # baseline: directly load the model
        # pad token:
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        model = AutoModelForCausalLM.from_pretrained(model_name)
        model.to(self.device)
        return model, tokenizer

    def initialize_draft_model(self, model_name: str):
        """
        Initialize a smaller, faster draft model with proper pad token.
        Uses lower precision and additional optimizations.
        """
        print(f"Loading draft model: {model_name}")
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        # TODO: Implement draft model initialization
        # 1. Set the pad token if it doesn't exist
        # 2. Load the model with appropriate settings for inference
        # 3. Enable any optimizations that might help with performance
        if not hasattr(self, 'target_model'):
            self.target_model, self.target_tokenizer = self.initialize_target_model(self.target_model_name)

        # pad token:
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        # baseline: directly load the model 
        model = AutoModelForCausalLM.from_pretrained(model_name)
        model.to(self.device)

        return model, tokenizer

    def generate_draft_tokens(self, input_ids: torch.Tensor, attention_mask: torch.Tensor,
                             num_speculative_tokens: int = 10) -> torch.Tensor:
        """
        Generate speculative tokens in one forward call using the draft model.

        Args:
            input_ids: Input token IDs (tensor of shape [1, seq_len]).
            attention_mask: Corresponding attention mask.
            num_speculative_tokens: Number of tokens to speculate.

        Returns:
            Tensor of shape [1, num_speculative_tokens] containing the draft tokens.
        """
        # TODO: Implement draft token generation
        # 1. Use the draft model to generate tokens
        # 2. Extract only the new tokens (not including the input)
        # 3. Return the newly generated tokens
        # baseline: directly generate tokens
        if not hasattr(self, 'draft_model'):
            self.draft_model, self.draft_tokenizer = self.initialize_draft_model(self.draft_model_name)
        
        max_tokens = num_speculative_tokens + input_ids.shape[-1]
        generation_config = GenerationConfig(
            max_new_tokens=num_speculative_tokens,
            do_sample=True,
            # temperature=0.3,
            pad_token_id=self.draft_tokenizer.pad_token_id,
            eos_token_id=self.draft_tokenizer.eos_token_id,
            bos_token_id=self.draft_tokenizer.bos_token_id,
        )
        self.draft_model.generation_config = generation_config
        draft = self.draft_model.generate(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            use_cache=True,
            temperature=0.34,  # 降低 temperature，生成更确定的 token
            top_k=50,         # 限制 token 选择范围
            top_p=0.85,        # 降低多样性，避免 draft model 生成错误 token
        )

        return draft[:, input_ids.shape[-1]:]

    def verify_tokens_vectorized(self, input_ids: torch.Tensor, draft_tokens: torch.Tensor,
                               attention_mask: torch.Tensor) -> Tuple[List[int], int]:
        """
        Vectorized verification: verify all draft tokens in one forward pass using the target model.

        Args:
            input_ids: The current input token IDs (shape [1, L]).
            draft_tokens: Draft tokens from the draft model (shape [1, k]).
            attention_mask: The current attention mask for input_ids.

        Returns:
            accepted_tokens: List of accepted token IDs.
            accepted_position: Index of the first rejected token (if all accepted, equals draft_tokens.shape[1]).
        """
        # TODO: Implement efficient verification of draft tokens
        # 1. Run target model on input_ids concatenated with draft_tokens
        # 2. Extract the logits for positions where draft tokens would be predicted
        # 3. Compare target model predictions with draft tokens
        # 4. Determine how many consecutive tokens were accepted before first mismatch
        target_input_ids = torch.cat([input_ids, draft_tokens], dim=-1)
        target_attention_mask = torch.cat([attention_mask, torch.ones_like(draft_tokens)], dim=-1)
        target_input_ids = target_input_ids.to(self.device)
        target_attention_mask = target_attention_mask.to(self.device)
        with torch.no_grad():
            logits = self.target_model(target_input_ids, attention_mask=target_attention_mask).logits
        
        draft_start_idx = input_ids.shape[-1]-1
        target_draft_logits = logits[:, draft_start_idx:-1, :]

        # get target tokens
        target_predicted_tokens = target_draft_logits.argmax(dim=-1)

        # matches tokens
        match_mask = (target_predicted_tokens != draft_tokens).int() # 0 if match, 1 if mismatch

        if match_mask.sum() != 0:
            accept_position = torch.argmax(match_mask, dim=-1)
        else:
            accept_position = draft_tokens.shape[-1]

        accepted_tokens = draft_tokens[:,:accept_position]
        return accepted_tokens, accept_position
    
    def verify_tokens_cached(self, input_ids: torch.Tensor, draft_tokens: torch.Tensor,
                             attention_mask: torch.Tensor, past_key_values=None):
        """
        Vectorized verification: verify all draft tokens in one forward pass using the target model.
        """
        input_ids = input_ids.long()
        attention_mask = attention_mask.long()
        draft_tokens = draft_tokens.long()

        target_input_ids = torch.cat([input_ids, draft_tokens], dim=-1)
        target_attention_mask = torch.cat([attention_mask, torch.ones_like(draft_tokens)], dim=-1)

        with torch.no_grad():
            outputs = self.target_model(
                target_input_ids,
                attention_mask=target_attention_mask,
                # past_key_values=past_key_values,  # 传入历史 KV Cache
                use_cache=True  # 让模型存储 KV Cache
            )
        
        logits = outputs.logits

        draft_start_idx = input_ids.shape[-1]-1
        target_draft_logits = logits[:, draft_start_idx:-1, :]
        target_predicted_tokens = target_draft_logits.argmax(dim=-1)

        match_mask = (target_predicted_tokens != draft_tokens).int()  # 1 if mismatch, 0 if match

        if match_mask.sum() > 0:
            accept_position = torch.argmax(match_mask, dim=-1)
        else:
            accept_position = draft_tokens.shape[-1]

        accepted_tokens = draft_tokens[:, :accept_position]
        # new_past_key_values = outputs.past_key_values  
        new_past_key_values = tuple(
            (layer_K[:, :, :input_ids.shape[-1] + accept_position, :], layer_V[:, :, :input_ids.shape[-1] + accept_position, :])
            for layer_K, layer_V in outputs.past_key_values
        )

        return accepted_tokens, accept_position, new_past_key_values  

    def speculative_decode(self, prompt: str, max_tokens: int = 100,
                          num_speculative_tokens: int = 15, use_cached: bool = False) -> str:
        """
        Main speculative decoding algorithm with vectorized verification.

        Args:
            prompt: Input text.
            max_tokens: Maximum number of tokens to generate (excluding prompt).
            num_speculative_tokens: Number of tokens to speculate per iteration.

        Returns:
            Generated text.
        """
        # Tokenize prompt
        inputs = self.target_tokenizer(prompt, return_tensors="pt", padding=True)
        input_ids = inputs["input_ids"].to(self.device)
        attention_mask = inputs["attention_mask"].to(self.device)
        prompt_length = input_ids.shape[1]

        # Initialize counters for performance tracking
        total_tokens_generated = prompt_length
        total_draft_tokens_proposed = 0
        total_draft_tokens_accepted = 0
        start_time = time.time()

        # TODO: Implement the core speculative decoding loop
        # 1. Generate draft tokens using the draft model
        # 2. Verify draft tokens using the target model
        # 3. Accept verified tokens and append to the sequence
        # 4. For rejected tokens or if all tokens are accepted, generate a new token with the target model
        # 5. Stop when max_tokens is reached or an EOS token is generated
        past_key_values = None
        
        while total_tokens_generated - prompt_length < max_tokens and input_ids[0, -1] != self.target_tokenizer.eos_token_id:   
            # draft generation:
            draft_ids = self.generate_draft_tokens(input_ids, attention_mask, num_speculative_tokens)
            total_draft_tokens_proposed += num_speculative_tokens

            # verification:
            if use_cached:
                accepted_draft_ids, accepted_position, new_past_key_values = self.verify_tokens_cached(input_ids, draft_ids, attention_mask, past_key_values)
                past_key_values = new_past_key_values
                total_draft_tokens_accepted += accepted_draft_ids.shape[-1]

                input_ids = torch.cat([input_ids, accepted_draft_ids], dim=-1)
                attention_mask = torch.cat([attention_mask, torch.ones_like(accepted_draft_ids)], dim=-1)

                if accepted_position == num_speculative_tokens or accepted_position == 0:
                    input_ids = self.target_model.generate(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        max_new_tokens=1,
                        do_sample=False,
                        pad_token_id=self.target_tokenizer.pad_token_id,
                        use_cache=True,
                        # past_key_values=past_key_values
                    )


                    # input_ids = torch.cat([input_ids, new_token], dim=-1)
                    attention_mask = torch.cat([attention_mask, torch.ones_like(torch.tensor([[1]]).to(self.device))], dim=-1)

            else:
                accepted_draft_ids, accepted_position = self.verify_tokens_vectorized(input_ids, draft_ids, attention_mask)
                total_draft_tokens_accepted += accepted_draft_ids.shape[-1]
                # print("accepted_draft_ids", accepted_draft_ids)

                # print("before concatenating accepted_draft_ids:", input_ids.shape[-1])

                # append accepted tokens to the sequence
                input_ids = torch.cat([input_ids, accepted_draft_ids], dim=-1)
                attention_mask = torch.cat([attention_mask, torch.ones_like(accepted_draft_ids)], dim=-1)
                
                # print("Before concatenating new token:", input_ids.shape[-1])
                # if all accepted or rejected, generate a new token with the target model
                if accepted_position == num_speculative_tokens or accepted_position == 0:
                    # print("Start generating a new token!!!!")
                    # all accepted, generate a new token with the target model
                    input_ids = self.target_model.generate(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        max_new_tokens=1,
                        do_sample=False,
                        pad_token_id=self.target_tokenizer.pad_token_id
                    )
                    # print("new_token", input_ids)
                    attention_mask = torch.cat([attention_mask, torch.tensor([[1]]).to(self.device)], dim=-1)

                # update counters
            total_tokens_generated = input_ids.shape[-1]

        # Calculate performance metrics
        elapsed_time = time.time() - start_time
        acceptance_rate = total_draft_tokens_accepted / total_draft_tokens_proposed if total_draft_tokens_proposed > 0 else 0

        print(f"Generated {total_tokens_generated - prompt_length} tokens in {elapsed_time:.2f} seconds")
        print(f"Tokens per second: {(total_tokens_generated - prompt_length) / elapsed_time:.2f}")
        print(f"Draft token acceptance rate: {acceptance_rate:.2%}")

        return self.target_tokenizer.decode(input_ids[0], skip_special_tokens=True)

    def benchmark(self, prompt: str, max_tokens: int = 100,
                  num_runs: int = 3, compare_baseline: bool = True, use_cached: bool = False, num_speculative_tokens: int = 15) -> Dict:
        """
        Benchmark the speculative decoder against baseline decoding.

        Args:
            prompt: Input text.
            max_tokens: Maximum number of tokens to generate.
            num_runs: Number of benchmark runs.
            compare_baseline: Whether to compare with baseline (non-speculative) decoding.

        Returns:
            Dictionary with benchmark results.
        """
        results = {
            "speculative": {"times": [], "tokens_per_second": []},
            "baseline": {"times": [], "tokens_per_second": []} if compare_baseline else None
        }

        # Benchmark speculative decoding.
        for _ in range(num_runs):
            start_time = time.time()
            output = self.speculative_decode(prompt, max_tokens=max_tokens, use_cached=use_cached, num_speculative_tokens=num_speculative_tokens)
            elapsed = time.time() - start_time
            prompt_len = len(self.target_tokenizer(prompt)["input_ids"])
            output_tokens = len(self.target_tokenizer.encode(output)) - prompt_len
            tps = output_tokens / elapsed
            results["speculative"]["times"].append(elapsed)
            results["speculative"]["tokens_per_second"].append(tps)

        # Benchmark baseline decoding.
        if compare_baseline:
            for _ in range(num_runs):
                inputs = self.target_tokenizer(prompt, return_tensors="pt", padding=True)
                input_ids = inputs["input_ids"].to(self.device)
                attention_mask = inputs["attention_mask"].to(self.device)
                start_time = time.time()
                with torch.no_grad():
                    output_ids = self.target_model.generate(
                        input_ids,
                        attention_mask=attention_mask,
                        max_length=input_ids.shape[1] + max_tokens,
                        do_sample=False,
                        pad_token_id=self.target_tokenizer.pad_token_id
                    )
                elapsed = time.time() - start_time
                output_tokens = output_ids.shape[1] - input_ids.shape[1]
                tps = output_tokens / elapsed
                results["baseline"]["times"].append(elapsed)
                results["baseline"]["tokens_per_second"].append(tps)

        for method in results.keys():
            if results[method] is not None:
                avg_time = sum(results[method]["times"]) / num_runs
                avg_tps = sum(results[method]["tokens_per_second"]) / num_runs
                results[method]["avg_time"] = avg_time
                results[method]["avg_tokens_per_second"] = avg_tps

        if compare_baseline:
            speedup = results["baseline"]["avg_time"] / results["speculative"]["avg_time"]
            results["speedup"] = speedup
            results["latency_reduction"] = (1 - results["speculative"]["avg_time"] / results["baseline"]["avg_time"]) * 100
            # print(f"Speculative decoding speedup: {speedup:.2f}x")
            # print(f"Latency reduction: {results['latency_reduction']:.2f}%")

        return results

## Test

In [3]:
target_model_name = "EleutherAI/pythia-1.4b-deduped"  # Larger target model
draft_model_name = "EleutherAI/pythia-160m-deduped"   # Smaller draft model


# Initialize speculative decoder
decoder = SpeculativeDecoder(
    target_model_name=target_model_name,
    draft_model_name=draft_model_name,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# Test prompts
test_prompts = [
    "The future of Artificial Intelligence is",
    "Write a short story about a robot learning to feel emotions:",
    "Write the lyrics to the song 'Happy Birthday'."
]

# Run benchmark on test prompts
for i, prompt in enumerate(test_prompts):
    print(f"\nBenchmarking Prompt {i+1}:")
    print(f"Prompt: {prompt}")

    results = decoder.benchmark(
        prompt=prompt,
        max_tokens=100,
        num_runs=3,
        compare_baseline=True
    )

    print(f"Average speculative decoding time: {results['speculative']['avg_time']:.2f} seconds")
    print(f"Average speculative tokens per second: {results['speculative']['avg_tokens_per_second']:.2f}")

    if results["baseline"] is not None:
        print(f"Average baseline decoding time: {results['baseline']['avg_time']:.2f} seconds")
        print(f"Average baseline tokens per second: {results['baseline']['avg_tokens_per_second']:.2f}")
        print(f"Speedup: {results['speedup']:.2f}x")
        print(f"Latency reduction: {results['latency_reduction']:.2f}%")

Loading target model: EleutherAI/pythia-1.4b-deduped


The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


Loading draft model: EleutherAI/pythia-160m-deduped

Benchmarking Prompt 1:
Prompt: The future of Artificial Intelligence is
Generated 109 tokens in 6.01 seconds
Tokens per second: 18.15
Draft token acceptance rate: 46.67%
Generated 107 tokens in 5.11 seconds
Tokens per second: 20.93
Draft token acceptance rate: 46.19%
Generated 103 tokens in 4.13 seconds
Tokens per second: 24.92
Draft token acceptance rate: 56.97%
Average speculative decoding time: 5.09 seconds
Average speculative tokens per second: 21.22
Average baseline decoding time: 2.64 seconds
Average baseline tokens per second: 37.91
Speedup: 0.52x
Latency reduction: -93.06%

Benchmarking Prompt 2:
Prompt: Write a short story about a robot learning to feel emotions:
Generated 104 tokens in 9.41 seconds
Tokens per second: 11.06
Draft token acceptance rate: 22.82%
Generated 115 tokens in 7.13 seconds
Tokens per second: 16.12
Draft token acceptance rate: 35.00%
Generated 106 tokens in 6.65 seconds
Tokens per second: 15.94
Draft to

## Use cached KV Cache

In [4]:
target_model_name = "EleutherAI/pythia-1.4b-deduped"  # Larger target model
draft_model_name = "EleutherAI/pythia-160m-deduped"   # Smaller draft model


# Initialize speculative decoder
decoder = SpeculativeDecoder(
    target_model_name=target_model_name,
    draft_model_name=draft_model_name,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# Test prompts
test_prompts = [
    "The future of Artificial Intelligence is",
    "Write a short story about a robot learning to feel emotions:",
    "Write the lyrics to the song 'Happy Birthday'."
]

# Run benchmark on test prompts
for i, prompt in enumerate(test_prompts):
    print(f"\nBenchmarking Prompt {i+1}:")
    print(f"Prompt: {prompt}")

    results = decoder.benchmark(
        prompt=prompt,
        max_tokens=100,
        num_runs=3,
        compare_baseline=True,
        use_cached=True,
        num_speculative_tokens=12
    )

    print(f"Average speculative decoding time: {results['speculative']['avg_time']:.2f} seconds")
    print(f"Average speculative tokens per second: {results['speculative']['avg_tokens_per_second']:.2f}")

    if results["baseline"] is not None:
        print(f"Average baseline decoding time: {results['baseline']['avg_time']:.2f} seconds")
        print(f"Average baseline tokens per second: {results['baseline']['avg_tokens_per_second']:.2f}")
        print(f"Speedup: {results['speedup']:.2f}x")
        print(f"Latency reduction: {results['latency_reduction']:.2f}%")

Loading target model: EleutherAI/pythia-1.4b-deduped
Loading draft model: EleutherAI/pythia-160m-deduped

Benchmarking Prompt 1:
Prompt: The future of Artificial Intelligence is
Generated 109 tokens in 1.42 seconds
Tokens per second: 76.87
Draft token acceptance rate: 83.33%
Generated 105 tokens in 1.30 seconds
Tokens per second: 80.96
Draft token acceptance rate: 88.89%
Generated 105 tokens in 1.30 seconds
Tokens per second: 81.03
Draft token acceptance rate: 88.89%
Average speculative decoding time: 1.34 seconds
Average speculative tokens per second: 79.57
Average baseline decoding time: 1.48 seconds
Average baseline tokens per second: 67.42
Speedup: 1.11x
Latency reduction: 9.80%

Benchmarking Prompt 2:
Prompt: Write a short story about a robot learning to feel emotions:
Generated 112 tokens in 1.32 seconds
Tokens per second: 84.78
Draft token acceptance rate: 96.30%
Generated 112 tokens in 1.67 seconds
Tokens per second: 67.04
Draft token acceptance rate: 71.53%
Generated 110 token

## Bonus1
Different model pairs:

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
# Define model pair (choose the best combination)
target_model_name = "EleutherAI/pythia-6.9b"
draft_model_name = "EleutherAI/pythia-1.4b"


# Initialize speculative decoder
decoder = SpeculativeDecoder(
    target_model_name=target_model_name,
    draft_model_name=draft_model_name,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# Test prompts
test_prompts = [
    "The future of Artificial Intelligence is",
    "Write a short story about a robot learning to feel emotions:",
    "Write the lyrics to the song 'Happy Birthday'."
]

# Run benchmark on test prompts
for i, prompt in enumerate(test_prompts):
    print(f"\nBenchmarking Prompt {i+1}:")
    print(f"Prompt: {prompt}")

    results = decoder.benchmark(
        prompt=prompt,
        max_tokens=100,
        num_runs=3,
        compare_baseline=True
    )

    print(f"Average speculative decoding time: {results['speculative']['avg_time']:.2f} seconds")
    print(f"Average speculative tokens per second: {results['speculative']['avg_tokens_per_second']:.2f}")

    if results["baseline"] is not None:
        print(f"Average baseline decoding time: {results['baseline']['avg_time']:.2f} seconds")
        print(f"Average baseline tokens per second: {results['baseline']['avg_tokens_per_second']:.2f}")
        print(f"Speedup: {results['speedup']:.2f}x")
        print(f"Latency reduction: {results['latency_reduction']:.2f}%")

Loading target model: EleutherAI/pythia-6.9b


The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.40s/it]


Loading draft model: EleutherAI/pythia-1.4b

Benchmarking Prompt 1:
Prompt: The future of Artificial Intelligence is
Generated 112 tokens in 3.74 seconds
Tokens per second: 29.91
Draft token acceptance rate: 100.00%
Generated 112 tokens in 2.91 seconds
Tokens per second: 38.54
Draft token acceptance rate: 100.00%
Generated 114 tokens in 3.17 seconds
Tokens per second: 35.91
Draft token acceptance rate: 89.17%
Average speculative decoding time: 3.28 seconds
Average speculative tokens per second: 34.64
Average baseline decoding time: 4.14 seconds
Average baseline tokens per second: 24.18
Speedup: 1.26x
Latency reduction: 20.70%

Benchmarking Prompt 2:
Prompt: Write a short story about a robot learning to feel emotions:
Generated 112 tokens in 10.25 seconds
Tokens per second: 10.93
Draft token acceptance rate: 22.53%
Generated 112 tokens in 9.51 seconds
Tokens per second: 11.78
Draft token acceptance rate: 24.69%
Generated 112 tokens in 7.61 seconds
Tokens per second: 14.72
Draft token ac

## Bonus
Higher acceptance rate:
When we want higher acceptance rate, we can generate less speculative tokens.

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
# Define model pair (choose the best combination)
target_model_name = "EleutherAI/pythia-1.4b-deduped"  # Larger target model
draft_model_name = "EleutherAI/pythia-160m-deduped"   # Smaller draft model



# Initialize speculative decoder
decoder = SpeculativeDecoder(
    target_model_name=target_model_name,
    draft_model_name=draft_model_name,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# Test prompts
test_prompts = [
    "The future of Artificial Intelligence is",
    "Write a short story about a robot learning to feel emotions:",
    "Write the lyrics to the song 'Happy Birthday'."
]

# Run benchmark on test prompts
for i, prompt in enumerate(test_prompts):
    print(f"\nBenchmarking Prompt {i+1}:")
    print(f"Prompt: {prompt}")

    results = decoder.benchmark(
        prompt=prompt,
        max_tokens=100,
        num_runs=3,
        compare_baseline=True,
        num_speculative_tokens=5,
        use_cached=True
    )

    print(f"Average speculative decoding time: {results['speculative']['avg_time']:.2f} seconds")
    print(f"Average speculative tokens per second: {results['speculative']['avg_tokens_per_second']:.2f}")

    if results["baseline"] is not None:
        print(f"Average baseline decoding time: {results['baseline']['avg_time']:.2f} seconds")
        print(f"Average baseline tokens per second: {results['baseline']['avg_tokens_per_second']:.2f}")
        print(f"Speedup: {results['speedup']:.2f}x")
        print(f"Latency reduction: {results['latency_reduction']:.2f}%")

Loading target model: EleutherAI/pythia-1.4b-deduped
Loading draft model: EleutherAI/pythia-160m-deduped

Benchmarking Prompt 1:
Prompt: The future of Artificial Intelligence is
Generated 103 tokens in 1.52 seconds
Tokens per second: 67.66
Draft token acceptance rate: 94.44%
Generated 103 tokens in 1.51 seconds
Tokens per second: 68.27
Draft token acceptance rate: 94.44%
Generated 100 tokens in 1.49 seconds
Tokens per second: 67.31
Draft token acceptance rate: 92.22%
Average speculative decoding time: 1.51 seconds
Average speculative tokens per second: 67.25
Average baseline decoding time: 1.50 seconds
Average baseline tokens per second: 66.83
Speedup: 0.99x
Latency reduction: -0.69%

Benchmarking Prompt 2:
Prompt: Write a short story about a robot learning to feel emotions:
Generated 105 tokens in 1.67 seconds
Tokens per second: 62.92
Draft token acceptance rate: 86.00%
Generated 104 tokens in 1.59 seconds
Tokens per second: 65.36
Draft token acceptance rate: 90.53%
Generated 105 toke